# Import related functions

In [1]:
import cobra
#from script.ECMpy_function import *
import sys
sys.path.append(r'./script/')
from ECMpy_function import *

# Input and output files

In [2]:
ecModel_file="./model/iML1515_irr_enz_constraint_adj.json"
obj='EX_trp__L_e' # EX_lys_L_e
substrate = 'EX_glc__D_e'
substrate_con=10
biomass_id = 'BIOMASS_Ec_iML1515_core_75p37M'
biomass_min=0.1


# Directly determining targets based on enzyme abundance

In [3]:
enzcost = pd.DataFrame()
fluxes_outfile = './analysis/ECMpy_solution_%s_%.2g_pfba.csv'%(obj,substrate_con)

enz_model=get_enzyme_constraint_model(ecModel_file)
enz_model.reactions.get_by_id(substrate).bounds=(-substrate_con,0)#EX_glc_e EX_fru_e EX_sucr_e EX_inost_e EX_lac_D_e EX_ac_e
enz_model.reactions.get_by_id('%s_reverse'%substrate).bounds=(0,0)
enz_model.reactions.get_by_id(biomass_id).bounds=(biomass_min,biomass_min)
enz_model.objective=obj
enz_model_pfba_solution = cobra.flux_analysis.pfba(enz_model)

enzcost = get_fluxes_detail_in_model(enz_model,enz_model_pfba_solution,fluxes_outfile,ecModel_file)
enzcost['enz_ratio'] = enzcost['E']/np.sum(enzcost['E'])
enzcost = enzcost.sort_values('enz_ratio', ascending=False)
enzcost_select = enzcost[enzcost['enz_ratio']>0.01]
enzcost_select

,fluxes,kcat_MW,E,equ,ec-code,enz_ratio
ATPS4rpp_num2,26.914001,285.026853,0.094426,adp_c + 4.0 h_p + pi_c --> atp_c + h2o_c + 3.0...,3.6.3.14,0.415974
PSERT,3.307705,112.661187,0.029360,3php_c + glu__L_c --> akg_c + pser__L_c,2.6.1.52,0.129338
ANPRT,3.141387,139.264990,0.022557,anth_c + prpp_c --> ppi_c + pran_c,2.4.2.18,0.099370
PFL_num3,6.207842,403.714736,0.015377,coa_c + pyr_c --> accoa_c + for_c,2.3.1.54,0.067739
LEUTAi_num2,0.045053,3.307456,0.013622,4mop_c + glu__L_c --> akg_c + leu__L_c,"2.6.1.42,2.6.1.6,2.6.1.67",0.060007
GLUDy_reverse,3.991100,335.933801,0.011881,akg_c + h_c + nadph_c + nh4_c --> glu__L_c + h...,NaN,0.052338
KARA1_reverse,0.087437,8.489153,0.010300,alac__S_c + h_c + nadph_c --> 23dhmb_c + nadp_c,NaN,0.045374
IGPS,3.141387,487.351491,0.006446,2cpr5p_c + h_c --> 3ig3p_c + co2_c + h2o_c,4.1.1.48,0.028396
TALA_num1,1.586907,664.414095,0.002388,g3p_c + s7p_c --> e4p_c + f6p_c,2.2.1.2,0.010522


#  Determine targets based on the fold changes of enzyme cost 

In [4]:
biomass_max=0.4
FC_threshold = 1.5
enzcost_diff_file = './analysis/enzcost_diff.csv'
fluxes_outfile = './analysis/ECMpy_solution_%s_%.2g_pfba.csv'%(obj,substrate_con)

enzcost_diff = get_enz_foldchange(ecModel_file,obj,substrate,substrate_con,biomass_id,biomass_min,biomass_max,FC_threshold,fluxes_outfile, enzcost_diff_file)
enzcost_diff_select = enzcost_diff[enzcost_diff['E_μ0.1']>0.0001]
enzcost_diff_select

,E_μ0.1,E_μ0.4,log2_foldchange(max/min),log2_foldchange(min/max),type,foldchange(max/min),equ,ec-code
PAPSR2_num2,0.000128,5.113764e-04,2.000000,-2.000000,weaken_target,4.000000,grxrd_c + paps_c --> grxox_c + 2.0 h_c + pap_c...,1.8.4.8
ACOTA_reverse_num2,0.000231,9.258608e-04,2.000000,-2.000000,weaken_target,4.000000,acg5sa_c + glu__L_c --> acorn_c + akg_c,NaN
HSDy_reverse_num1,0.000983,3.932984e-03,2.000000,-2.000000,weaken_target,4.000000,aspsa_c + h_c + nadph_c --> hom__L_c + nadp_c,NaN
KARA1_reverse,0.010300,4.119926e-02,2.000000,-2.000000,weaken_target,4.000000,alac__S_c + h_c + nadph_c --> 23dhmb_c + nadp_c,NaN
IPPMIa_reverse,0.000283,1.132100e-03,2.000000,-2.000000,weaken_target,4.000000,2ippm_c + h2o_c --> 3c2hmp_c,NaN
AIRC3_reverse,0.000113,4.536625e-04,2.000000,-2.000000,weaken_target,4.000000,5caiz_c --> 5aizc_c,NaN
IPPMIb_reverse,0.000283,1.132100e-03,2.000000,-2.000000,weaken_target,4.000000,3c3hmp_c --> 2ippm_c + h2o_c,NaN
IMPC_reverse,0.000178,7.121831e-04,2.000000,-2.000000,weaken_target,4.000000,fprica_c --> h2o_c + imp_c,NaN
LEUTAi_num2,0.013622,5.448672e-02,2.000000,-2.000000,weaken_target,4.000000,4mop_c + glu__L_c --> akg_c + leu__L_c,"2.6.1.42,2.6.1.6,2.6.1.67"
AICART,0.000120,4.813187e-04,2.000000,-2.000000,weaken_target,4.000000,10fthf_c + aicar_c --> fprica_c + thf_c,2.1.2.3


# Determine targets based on FESOF

In [5]:
model=get_enzyme_constraint_model(ecModel_file)

FSEOF_file = './analysis/FESEOF_gene.csv'
[FSEOFdf_detail,FSEOFdf_done,FESEOF_gene] = run_FSEOF(model,substrate,substrate_con,biomass_id,obj,FSEOF_file)
FSEOFdf_detail.head(10)

./script/ECMpy_function.py:2523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  always_down['regulation'] = 'down'
./script/ECMpy_function.py:2526: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  always_up['regulation'] = 'up'
./script/ECMpy_function.py:2539: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  FSEOFdf_g['genes'] = FSEOFdf_g['genes'].str.replace

,genes,growth = 0.2,FC growth = 0.2,growth = 0.22,FC growth = 0.22,growth = 0.24,FC growth = 0.24,growth = 0.25,FC growth = 0.25,growth = 0.27,...,growth = 0.34,FC growth = 0.34,growth = 0.36,FC growth = 0.36,WT,GPR,FC_mean,regulation,check,reactions
75,b1263,2.194427,96.190581,1.982141,86.885220,1.767434,77.473745,1.659719,72.752173,1.443458,...,0.686306,30.083548,0.469977,20.600973,0.022813,b1263 and b1264,36.871785,up,yes,"[ANPRT, ANS]"
416,b1260,2.194427,96.190581,1.982141,86.885220,1.767434,77.473745,1.659719,72.752173,1.443458,...,0.686306,30.083548,0.469977,20.600973,0.022813,b1260 and b1261,36.871785,up,yes,"[TRPS2, TRPS3, TRPS1]"
74,b1263,2.194427,96.190581,1.982141,86.885220,1.767434,77.473745,1.659719,72.752173,1.443458,...,0.686306,30.083548,0.469977,20.600973,0.022813,b1263,36.871785,up,yes,"[ANPRT, ANS]"
417,b1261,2.194427,96.190581,1.982141,86.885220,1.767434,77.473745,1.659719,72.752173,1.443458,...,0.686306,30.083548,0.469977,20.600973,0.022813,b1260 and b1261,36.871785,up,yes,"[TRPS2, TRPS3, TRPS1]"
76,b1264,2.194427,96.190581,1.982141,86.885220,1.767434,77.473745,1.659719,72.752173,1.443458,...,0.686306,30.083548,0.469977,20.600973,0.022813,b1263 and b1264,36.871785,up,yes,[ANS]
350,b1262,2.194427,96.190581,1.982141,86.885220,1.767434,77.473745,1.659719,72.752173,1.443458,...,0.686306,30.083548,0.469977,20.600973,0.022813,b1262,36.871785,up,yes,"[PRAIi, IGPS]"
231,b1262,2.194427,96.190581,1.982141,86.885220,1.767434,77.473745,1.659719,72.752173,1.443458,...,0.686306,30.083548,0.469977,20.600973,0.022813,b1262,36.871785,up,yes,"[PRAIi, IGPS]"
204,b0875,39.979146,1.692937,36.745743,1.556017,33.167997,1.404516,32.338863,1.369406,31.580777,...,28.926293,1.224899,28.167869,1.192783,23.615256,b0875,15.110857,up,yes,"[H2Otpp_reverse, H2Otpp]"
203,b3875,36.471817,1.560115,34.647743,1.482088,32.754616,1.401108,32.190856,1.376993,31.420929,...,28.725003,1.228738,27.954738,1.195789,23.377652,b3875,15.008271,up,yes,"[DHPStex_reverse, H2Otex_reverse_num4, H2Otex_..."
120,b0429,24.413395,4.767069,23.251614,4.540215,21.992884,4.294430,20.675285,4.037149,17.786598,...,7.855033,1.533809,5.017443,0.979729,5.121258,b0429 and b0432 and b0431 and b0430,9.922184,up,yes,[CYTBO3_4pp]


In [6]:
FESEOF_gene.head(10)

,genes,regulation,reactions,check
75,b1263,up,"[ANPRT, ANS]",yes
416,b1260,up,"[TRPS2, TRPS3, TRPS1]",yes
74,b1263,up,"[ANPRT, ANS]",yes
417,b1261,up,"[TRPS2, TRPS3, TRPS1]",yes
76,b1264,up,[ANS],yes
350,b1262,up,"[PRAIi, IGPS]",yes
231,b1262,up,"[PRAIi, IGPS]",yes
204,b0875,up,"[H2Otpp_reverse, H2Otpp]",yes
203,b3875,up,"[DHPStex_reverse, H2Otex_reverse_num4, H2Otex_...",yes
120,b0429,up,[CYTBO3_4pp],yes


In [7]:
FSEOFdf_done.head(10)

,growth = 0.2,FC growth = 0.2,growth = 0.22,FC growth = 0.22,growth = 0.24,FC growth = 0.24,growth = 0.25,FC growth = 0.25,growth = 0.27,FC growth = 0.27,...,FC growth = 0.33,growth = 0.34,FC growth = 0.34,growth = 0.36,FC growth = 0.36,WT,GPR,FC_mean,regulation,GPR1
id,,,,,,,,,,,,,,,,,,,,,
PRAIi,2.194427,96.190581,1.982141,86.885220,1.767434,77.473745,1.659719,72.752173,1.443458,63.272559,...,34.824835,0.686306,30.083548,0.469977,20.600973,0.022813,b1262,36.871785,up,b1262
TRPS1,2.194427,96.190581,1.982141,86.885220,1.767434,77.473745,1.659719,72.752173,1.443458,63.272559,...,34.824835,0.686306,30.083548,0.469977,20.600973,0.022813,b1260 and b1261,36.871785,up,b1260 or b1261
ANPRT,2.194427,96.190581,1.982141,86.885220,1.767434,77.473745,1.659719,72.752173,1.443458,63.272559,...,34.824835,0.686306,30.083548,0.469977,20.600973,0.022813,b1263,36.871785,up,b1263
ANS,2.194427,96.190581,1.982141,86.885220,1.767434,77.473745,1.659719,72.752173,1.443458,63.272559,...,34.824835,0.686306,30.083548,0.469977,20.600973,0.022813,b1263 and b1264,36.871785,up,b1263 or b1264
IGPS,2.194427,96.190581,1.982141,86.885220,1.767434,77.473745,1.659719,72.752173,1.443458,63.272559,...,34.824835,0.686306,30.083548,0.469977,20.600973,0.022813,b1262,36.871785,up,b1262
H2Otpp_reverse,39.979146,1.692937,36.745743,1.556017,33.167997,1.404516,32.338863,1.369406,31.580777,1.337304,...,1.240956,28.926293,1.224899,28.167869,1.192783,23.615256,b0875,15.110857,up,b0875
H2Otex_reverse_num4,36.471817,1.560115,34.647743,1.482088,32.754616,1.401108,32.190856,1.376993,31.420929,1.344058,...,1.245212,28.725003,1.228738,27.954738,1.195789,23.377652,b3875,15.008271,up,b3875
CYTBO3_4pp,24.413395,4.767069,23.251614,4.540215,21.992884,4.294430,20.675285,4.037149,17.786598,3.473091,...,1.810850,7.855033,1.533809,5.017443,0.979729,5.121258,b0429 and b0432 and b0431 and b0430,9.922184,up,b0429 or b0432 or b0431 or b0430
ATPS4rpp_num2,23.358835,1.625301,21.791138,1.516221,20.006023,1.392013,19.493032,1.356319,18.817127,1.309290,...,1.168005,16.448152,1.144457,15.771301,1.097362,14.372009,b3732 and b3735 and b3731 and b3733 and b3734 ...,9.270093,up,b3732 or b3735 or b3731 or b3733 or b3734 or b...
